In [1]:

import tensorflow as tf
from tensorflow import keras
import numpy as np


np.random.seed(42)
tf.random.set_seed(42)

import matplotlib.pyplot as plt

In [19]:
filepath =r"C:\Users\hp\Desktop\ML Models\RNNs\shakespeare.txt"
with open(filepath) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])



First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [20]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [23]:
print(tokenizer.texts_to_sequences(["First"]))
print(tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]]))


[[20, 6, 9, 8, 3]]
['f i r s t']


In [25]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count
print(max_id, dataset_size)

39 1115394


In [29]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
print(encoded)

[19  5  8 ... 20 26 10]


In [31]:
#taking 90% of dataset as training data
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
#above "from_tensor_slices"  creates a seperate tensor dataset from each element of the tensor encoded


In [32]:
dataset = dataset.repeat().window(101,shift=1,drop_remainder=True)   
#repeat() method basically continues the data until teh window is filled
#exammple: if data is [1,2,3] and window is 4 then repeat() will make it [1,2,3,1] [2,3,1,2]

In [33]:
dataset = dataset.flat_map(lambda window: window.batch(101))
 #It’s a nested dataset, analogous to a list of lists. However, we cannot use a nested dataset directly for training, as our model will expect tensors as input, not datasets. So, we must call the flat_map() method: it converts a nested dataset into a flat dataset.


In [34]:
np.random.seed(42)
tf.random.set_seed(42)

In [35]:

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [36]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [37]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [38]:
#Dropout is a regularization technique where a certain proportion of randomly selected neurons are "dropped out" 
#or ignored during training. This means that their contribution to the next layer is temporarily removed.
# in this case the drouput rate is 0.2 (its normaly between 0-1) 

#Recurrent Dropout is a variant of dropout specifically designed for recurrent neural networks. 
#In RNNs like GRUs or LSTMs, dropout is applied not only to the input units of a layer but also to 
#the recurrent connections within the layer.


model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [39]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                   epochs=10)
model.save("shakespeare.h5")

Epoch 1/10
  879/31370 [..............................] - ETA: 1:54:23 - loss: 2.1771

KeyboardInterrupt: 